In [ ]:
%load_ext autoreload
%autoreload 2

import img_utils as im
import numpy as np

In [ ]:
from patchmatch.patchmatch import PatchMatch
from netty import model_vgg, build_utils, vgg_utils

In [ ]:
def reconstruct_vgg(img,offsets,l,ksize,stride_1,stride_2):
    pm,pn = offsets.shape[:2]
    m_,n_ = ((pm-1)*stride_1+ksize,(pn-1)*stride_1+ksize)
    fullsize = vgg_utils.get_location([(m_//stride_1-1)*stride_1+ksize-1,(n_//stride_1-1)*stride_1+ksize-1],l)
    m = fullsize[0][1]
    n = fullsize[1][1]
    out = np.zeros((m,n,3),dtype=np.int32)
    for i in range(0,pm-1):
        for j in range(0,pn-1):
            begin = vgg_utils.get_location([i*stride_1,j*stride_1],l)
            end = vgg_utils.get_location([i*stride_1+ksize-1,j*stride_1+ksize-1],l)
            i_,j_ = offsets[i][j] * stride_2
            begin_ = vgg_utils.get_location([i_,j_],l)
            end_ = vgg_utils.get_location([i_+ksize-1,j_+ksize-1],l)

            patch = img[begin_[0][0]:end_[0][1],begin_[1][0]:end_[1][1]]
            out[begin[0][0]:end[0][1],begin[1][0]:end[1][1]] = patch
    return out

In [ ]:
img = im.size(im.load("data/me4.jpg"),(512,512))
# img2 = im.size(im.load("data/ppunk.jpg"),(512//16,512//16))
img2 = im.load("data/icon.jpg")

In [ ]:
gmodel = model_vgg.load_model(model="vgg19")

In [ ]:
for i in range(4,8):
#     gmodel = model_vgg.load_model(model="vgg19")
    model = build_utils.extract_layers(gmodel,[i])

    f1 = model.predict(np.float32([img]))[0]
    f2 = model.predict(np.float32([img2]))[0]
    
    ksize = 1

    k = vgg_utils.get_location([0,0],i)[0][1] // vgg_utils.get_location([1,0],i)[0][0]
    
    print(k)
    
    p = PatchMatch()
    p.setup(f1,f2,ksize=ksize,stride_1=k+ksize-1,stride_2=1)

    p.match(4)

    out = reconstruct_vgg(img2,p.offsets,i,ksize,k+ksize-1,1)
    im.show(out)
    im.save(out,"data/out/patcher/d_"+str(i)+".jpg")